In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Retinoblastoma/GSE59983'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression profiling of primary human retinoblastoma"
!Series_summary	"Background"
!Series_summary	"Retinoblastoma is a pediatric eye cancer associated with RB1 loss or MYCN amplification (RB1+/+MYCNA). There are controversies concerning the existence of molecular subtypes within RB1-/- retinoblastoma. To test whether these molecular subtypes exist, we performed molecular profiling."
!Series_summary	""
!Series_summary	"Methods"
!Series_summary	"Genome-wide mRNA expression profiling was performed on 76 primary human retinoblastomas. Expression profiling was complemented by genome-wide DNA profiling and clinical, histopathological, and ex vivo drug sensitivity data."
!Series_summary	""
!Series_summary	"Findings"
!Series_summary	"RNA and DNA profiling identified major variability between retinoblastomas. While gene expression differences between RB1+/+MYCNA and RB1-/- tumors seemed more dichotomous, differences within the RB1-/- tumors were grad

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_age = None  # define the functions when applicable

# Based on the dataset description, gene expression data is available
is_gene_available = True

# No information about retinoblastoma, age, or gender from sample characteristics dictionary
# Assuming data is not available
trait_row = age_row = gender_row = None

# Since data is not explicitly given for Retinoblastoma, age and gender, we'll define dummy converters for now
def convert_trait(value):
    return None

def convert_age(value):
    return None

def convert_gender(value):
    return None

# Saving cohort information
save_cohort_info('GSE59983', './preprocessed/Retinoblastoma/cohort_info.json', is_gene_available, trait_row is not None)

# Skipping clinical feature extraction as trait_row is None
